### Types of object detections
##### Template Matching: Looking for exact copy of a smaller img in bigger img
##### Corner Detection: Looking for corner in img
##### Edge Detection: Looking for general edges in img
##### Grid Detection: Combine both concept to find grids
##### Contour Detection: Detect foreground from background
##### Feature matching

##### Watershed Algorithm: Allow us to segment foreground and background
##### Facial and eye detection: Haar Cascades to detect face and eyes but no recognition which need deep learning



In [4]:
# Template matching

In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline